In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
'''La tabla dinámica toma los datos de las columnas como entrada y las agrupa en una tabla que proporciona un resumen multidimensional de los datos. 
Es esencialmente una versión multidimensional de la agregación con GroupBy.'''

'La tabla dinámica toma los datos de las columnas como entrada y las agrupa en una tabla que proporciona un resumen multidimensional de los datos. \nEs esencialmente una versión multidimensional de la agregación con GroupBy.'

In [3]:
## Descarguemos una base de datos del Titanic

titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
## Vemos los posibles valores para algunas columnas

titanic.parch.unique()

array([0, 1, 2, 5, 3, 4, 6])

In [5]:
titanic.embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [6]:
titanic.who.unique()

array(['man', 'woman', 'child'], dtype=object)

In [7]:
titanic.shape

(891, 15)

In [8]:
## Hagamos algunas agrupaciones

titanic.groupby('pclass')['survived'].sum()  ## Vemos cuántas personas sobrevivieron por clase

pclass
1    136
2     87
3    119
Name: survived, dtype: int64

In [9]:
titanic.groupby('sex')[['survived']].sum()  ## Sobrevivientes de acuerdo al sexo

,survived
sex,
female,233
male,109


In [10]:
titanic.groupby('embarked')[['survived']].sum()  ## Sobrevivientes de acuerdo al embarque

,survived
embarked,
C,93
Q,30
S,217


In [11]:
## Vemos cuántos sobrevientes hubo por sexo y clase

titanic.groupby(['sex', 'class'])['survived'].aggregate('sum').unstack()  

class,First,Second,Third
sex,,,
female,91,70,72
male,45,17,47


In [12]:
## Vemos cuántos sobrevientes hubo por sexo, adultosd o niños y clase

titanic.groupby(['sex', 'who', 'class'])['survived'].aggregate('sum').unstack()

class         First  Second  Third
sex    who                        
female child      2      10     16
       man        0       0      0
       woman     89      60     56
male   child      3       9      9
       man       42       8     38
       woman      0       0      0

In [13]:
## Este GroupBy bidimensional es lo suficientemente común como para que Pandas incluya una rutina conveniente 
## pivot_table que maneja este tipo de agregación multidimensional. Por defecto muestra la media

titanic.pivot_table('survived', index='sex', columns='class' , aggfunc= "sum")

class,First,Second,Third
sex,,,
female,91,70,72
male,45,17,47


In [14]:
## Agrupamos por dos columnas y aplicar una función correspondiente a cada una

titanic.pivot_table(index='sex', columns='class', aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

In [15]:
## La agrupación en tablas dinámicas se puede especificar con múltiples niveles. Podríamos estar interesados 
## en ver la edad como una tercera dimensión. Seccionaremos la edad usando la función pd.cut

edad = pd.cut(titanic['age'], [0 , 18, 50, 80])
titanic.pivot_table('survived' , ['sex', edad] , 'class' , aggfunc= 'sum')

class            First  Second  Third
sex    age                           
female (0, 18]      10      14     22
       (18, 50]     59      52     24
       (50, 80]     13       2      1
male   (0, 18]       4       9     11
       (18, 50]     31       5     27
       (50, 80]      5       1      0

In [16]:
## Podemos aplicar esta misma estrategia para trabajar con las columnas. Agreguemos información sobre la tarifa
## pagada usando pd.qcut para calcular automáticamente los cuantiles

fare = pd.qcut(titanic['fare'], 2)
multi = titanic.pivot_table('survived' , ['sex', edad], [ fare, 'class'] , aggfunc='sum')
multi.fillna(0)

fare            (-0.001, 14.454]              (14.454, 512.329]             
class                      First Second Third             First Second Third
sex    age                                                                  
female (0, 18]                 0      3    15                10     11     7
       (18, 50]                0     22    15                59     30     9
       (50, 80]                0      0     1                13      2     0
male   (0, 18]                 0      0     6                 4      9     5
       (18, 50]                0      4    22                31      1     5
       (50, 80]                0      1     0                 5      0     0

In [17]:
## El resultado es una agregación de cuatro dimensiones con índices jerárquicos.

multi.sum(level=0)

/tmp/ipykernel_17005/883586428.py:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  multi.sum(level=0)


fare   (-0.001, 14.454]              (14.454, 512.329]             
class             First Second Third             First Second Third
sex                                                                
female                0     25    31                82     43    16
male                  0      5    28                40     10    10

In [18]:
multi.sum(level=1)

/tmp/ipykernel_17005/2945875048.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  multi.sum(level=1)


fare     (-0.001, 14.454]              (14.454, 512.329]             
class               First Second Third             First Second Third
age                                                                  
(0, 18]                 0      3    21                14     20    12
(18, 50]                0     26    37                90     31    14
(50, 80]                0      1     1                18      2     0

In [19]:
## Con la palabra clave axis podemos acceder a los niveles en las columnas

multi.sum(axis=0)

fare               class 
(-0.001, 14.454]   First       0
                   Second     30
                   Third      59
(14.454, 512.329]  First     122
                   Second     53
                   Third      26
dtype: int64

In [20]:
multi.sum(axis=1)

sex     age     
female  (0, 18]      46
        (18, 50]    135
        (50, 80]     16
male    (0, 18]      24
        (18, 50]     63
        (50, 80]      6
dtype: int64

In [21]:
multi.sum(axis=1 , level = 'fare')

/tmp/ipykernel_17005/2117990215.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  multi.sum(axis=1 , level = 'fare')


fare             (-0.001, 14.454]  (14.454, 512.329]
sex    age                                          
female (0, 18]                 18                 28
       (18, 50]                37                 98
       (50, 80]                 1                 15
male   (0, 18]                  6                 18
       (18, 50]                26                 37
       (50, 80]                 1                  5

In [22]:
## A veces es útil calcular totales a lo largo de cada grupo. Esto se puede hacer a través de la palabra clave de
## margin

titanic.pivot_table('survived', 'sex', columns='class', margins=True, aggfunc=sum)

class,First,Second,Third,All
sex,,,,
female,91,70,72,233
male,45,17,47,109
All,136,87,119,342
